In [3]:
import modin.pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.recommendation import ALS as newALS
from sklearn.metrics import mean_squared_error
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

In [1]:
path = "../../Downloads/ml-1m"

def split_data(path, sep):  
    df = pd.read_csv(f'{path}/ratings.dat', sep = sep, names=['user_id', 'movie_id', 'rating', 'timestamp']).sort_values('timestamp')
    start_ts = df['timestamp'].min()
    med_ts = df['timestamp'].quantile(.5)
    end_ts = df['timestamp'].max()
    train_df = df[df['timestamp'] <= med_ts]
    stream_df = df[df['timestamp'] > med_ts]
    '''
    seen_movies = set(train_df['movie_id'])
    print(len(seen_movies), len(set(stream_df['movie_id'])), len(stream_df))
    stream_df = stream_df.drop(stream_df[stream_df['movie_id'].map(lambda x: x not in seen_movies)].index)
    print(len(set(stream_df['movie_id'])), len(stream_df))
    seen_users = set(train_df['user_id'])
    num_stream_users = set(stream_df['user_id'])
    stream_df = stream_df.drop(stream_df[stream_df['user_id'].map(lambda x: x not in seen_users)].index)
    print(len(seen_users), len(num_stream_users), len(set(stream_df['user_id'])), len(stream_df))
    '''
    #train_df.to_csv(f'{path}/train.csv', header=True, index = False)
    #stream_df.to_csv(f'{path}/stream.csv', header=True, index = False)
    return start_ts, med_ts, end_ts

In [4]:
split_data(path, "::")

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

To request implementation, send an email to feature_requests@modin.org.


(956703932, 973018006.0, 1046454590)

In [5]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 14:26:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark_df = spark.read.options(delimiter=",", inferSchema=True, header=True).csv(f'{path}/train.csv')

In [7]:
spark_df.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|   6040|     858|     4|956703932|
|   6040|    2384|     4|956703954|
|   6040|     593|     5|956703954|
|   6040|    1961|     4|956703977|
|   6040|    2019|     5|956703977|
|   6040|    1419|     3|956704056|
|   6040|     213|     5|956704056|
|   6040|    3111|     5|956704056|
|   6040|     573|     4|956704056|
|   6040|    3505|     4|956704056|
|   6040|    1734|     2|956704081|
|   6040|    2503|     5|956704191|
|   6040|     919|     5|956704191|
|   6040|     912|     5|956704191|
|   6040|     527|     5|956704219|
|   6040|    1252|     5|956704257|
|   6040|     649|     5|956704257|
|   6040|     318|     4|956704257|
|   6040|    3289|     5|956704305|
|   6040|     759|     5|956704448|
+-------+--------+------+---------+
only showing top 20 rows



In [8]:
spark_df = spark_df.drop('timestamp')
spark_df.show()
new_model = newALS.train(ratings = spark_df.rdd, rank = 150, iterations=15, lambda_=.2)

+-------+--------+------+
|user_id|movie_id|rating|
+-------+--------+------+
|   6040|     858|     4|
|   6040|    2384|     4|
|   6040|     593|     5|
|   6040|    1961|     4|
|   6040|    2019|     5|
|   6040|    1419|     3|
|   6040|     213|     5|
|   6040|    3111|     5|
|   6040|     573|     4|
|   6040|    3505|     4|
|   6040|    1734|     2|
|   6040|    2503|     5|
|   6040|     919|     5|
|   6040|     912|     5|
|   6040|     527|     5|
|   6040|    1252|     5|
|   6040|     649|     5|
|   6040|     318|     4|
|   6040|    3289|     5|
|   6040|     759|     5|
+-------+--------+------+
only showing top 20 rows



22/03/15 11:47:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/15 11:47:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/03/15 11:47:20 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [9]:
spark_df.sort("user_id").show()

+-------+--------+------+
|user_id|movie_id|rating|
+-------+--------+------+
|   2783|     541|     4|
|   2783|    3927|     3|
|   2783|     750|     5|
|   2783|    1097|     3|
|   2783|     260|     4|
|   2783|     208|     1|
|   2783|    1199|     3|
|   2783|    2111|     3|
|   2783|    1214|     4|
|   2783|    1270|     4|
|   2783|     589|     5|
|   2783|    1210|     3|
|   2783|    1243|     3|
|   2783|    1200|     4|
|   2783|    1077|     5|
|   2783|     741|     5|
|   2783|    3702|     3|
|   2783|    1301|     5|
|   2783|    3471|     3|
|   2783|    2968|     5|
+-------+--------+------+
only showing top 20 rows



In [10]:
user_features = new_model.userFeatures().toDF().toPandas()
user_features = user_features.rename(columns={'_1': 'id', '_2': 'features'}).sort_values("id")
user_features

22/03/15 11:48:54 WARN BlockManager: Task 49 already completed, not releasing lock for rdd_321_0


,id,features
0,2783,"[-0.07469388097524643, -0.11229582130908966, -..."
1,2784,"[-0.08663611114025116, -0.010234715417027473, ..."
2,2785,"[-0.032865673303604126, -0.008891520090401173,..."
3,2786,"[-0.0562487356364727, -0.06679890304803848, -0..."
4,2787,"[-0.06586312502622604, -0.06487643718719482, -..."
...,...,...
3250,6036,"[-0.04556586593389511, -0.11425884813070297, 0..."
3251,6037,"[-0.05084821954369545, -0.07124742120504379, -..."
3252,6038,"[-0.004150833003222942, -0.046171560883522034,..."
3253,6039,"[-0.026292433962225914, -0.058327674865722656,..."


In [11]:
product_features = new_model.productFeatures().toDF().toPandas()
product_features = product_features.rename(columns={'_1': 'id', '_2': 'features'})

22/03/15 11:48:55 WARN BlockManager: Task 51 already completed, not releasing lock for rdd_322_0


In [12]:
product_features.sort_values('id').to_csv(f'{path}/movie_features.csv', index = False)
user_features.sort_values('id').to_csv(f'{path}/user_features.csv', index = False)

In [13]:
def get_feature_dict(features):
    feature_dict = dict()
    for row in features.itertuples():
        feature_dict[int(row.id)] = np.array(row.features)
    return feature_dict

In [14]:
user_dict = get_feature_dict(user_features)
movie_dict = get_feature_dict(product_features)  

In [15]:
new_user_dict = get_feature_dict(user_features)
new_movie_dict = get_feature_dict(product_features)  

In [24]:
stream_df = pd.read_csv(f'{path}/stream.csv')
stream_df['user_id'] = stream_df['user_id'].astype(int)

def predict_rating(row):
    if row.movie_id not in movie_dict:
        movie_dict[row.movie_id] = np.zeros(150)
    if row.user_id not in user_dict:
        return -1
        user_dict[row.user_id] = np.zeros(150)
        return -1
    movie_feature = movie_dict[row.movie_id]
    user_feature = user_dict[row.user_id]
    return user_feature.dot(movie_feature.T)

def predict_table(df):
    df['prediction'] = df.apply(predict_rating , axis=1)
    return df

In [13]:
og_df = pd.read_csv(f'{path}/train.csv')
og_users = og_df['user_id'].unique()
og_users

array([6040, 6039, 6038, ..., 2785, 2784, 2783])

In [25]:
stream_df['in_og'] = stream_df['user_id'].apply(lambda x: x in og_users)

In [27]:
cleaned_stream_df = stream_df[stream_df['in_og'] == False]

In [31]:
cleaned_stream_df

,user_id,movie_id,rating,timestamp,in_og
4,2782,2580,4,973018637,False
5,2782,356,5,973018637,False
6,2782,1265,4,973018656,False
7,2782,1097,3,973018670,False
8,2782,380,3,973018687,False
...,...,...,...,...,...
500045,1647,1646,3,1046322905,False
500046,229,3089,4,1046325739,False
500047,229,2839,5,1046325855,False
500048,424,2581,3,1046349099,False


In [34]:
num_rows = []
for user_id in cleaned_stream_df['user_id'].unique():
    user_df = cleaned_stream_df[cleaned_stream_df['user_id'] == user_id]
    num_rows.append(len(user_df))
num_rows

(apply_list_of_funcs pid=55555) 
(apply_list_of_funcs pid=55548) 


[60,
 284,
 40,
 171,
 106,
 875,
 350,
 29,
 277,
 83,
 178,
 287,
 47,
 146,
 80,
 26,
 102,
 211,
 354,
 25,
 47,
 483,
 68,
 20,
 29,
 129,
 239,
 23,
 100,
 505,
 378,
 178,
 200,
 90,
 326,
 139,
 57,
 197,
 138,
 439,
 108,
 35,
 145,
 32,
 105,
 399,
 101,
 24,
 121,
 99,
 45,
 280,
 157,
 23,
 67,
 88,
 84,
 35,
 205,
 84,
 23,
 34,
 38,
 117,
 117,
 24,
 20,
 25,
 78,
 25,
 68,
 38,
 30,
 32,
 37,
 23,
 25,
 22,
 48,
 92,
 41,
 114,
 294,
 100,
 43,
 34,
 20,
 63,
 72,
 136,
 134,
 241,
 23,
 101,
 135,
 23,
 21,
 196,
 219,
 257,
 110,
 223,
 214,
 342,
 42,
 38,
 229,
 44,
 119,
 20,
 54,
 23,
 282,
 115,
 218,
 32,
 94,
 828,
 367,
 156,
 42,
 98,
 47,
 161,
 31,
 22,
 132,
 76,
 29,
 41,
 116,
 49,
 215,
 107,
 100,
 24,
 23,
 282,
 21,
 454,
 71,
 318,
 21,
 33,
 521,
 56,
 63,
 59,
 278,
 37,
 41,
 89,
 54,
 272,
 203,
 485,
 22,
 142,
 306,
 54,
 37,
 315,
 156,
 93,
 25,
 82,
 50,
 61,
 30,
 22,
 43,
 429,
 46,
 202,
 228,
 62,
 50,
 188,
 68,
 43,
 29,
 91,
 229,
 27

In [35]:
np.average(np.asarray(num_rows))
# switch to 50-100 features

162.44057450628367

22/03/16 18:43:06 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 322445 ms exceeds timeout 120000 ms
22/03/16 18:43:06 WARN SparkContext: Killing executors is not supported by current scheduler.


In [17]:
prediction_df = predict_table(stream_df)

In [22]:
sans_new = prediction_df[prediction_df['prediction'] != -1]
mean_squared_error(sans_new['rating'], sans_new['prediction'])

0.9636635490536786

In [23]:
class FIFOScheduler:
    
    def __init__(self):
        self.queue = []
        
    def push(self, row):
        self.queue.append(row)
    
    def pop(self):
        if len(self.queue) == 0:
            return None
        return self.queue.pop()

In [20]:
class LeastUpdatedScheduler:
    
    def __init__():
        self.queue =

SyntaxError: invalid syntax (3544257431.py, line 4)

In [24]:
stream_df['timestamp'].max() - stream_df['timestamp'].min(), len(stream_df)

(73436566, 500104)

In [55]:
class SGDUpdater:
    
    def __init__(self, user_features, product_features, learning_rate, u_reg, m_reg, update_movies = True):
        self.user_dict = get_feature_dict(user_features)
        self.movie_dict = get_feature_dict(product_features) 
        self.l = learning_rate
        self.u_reg = u_reg
        self.m_reg = m_reg
        self.update_movies = update_movies
    
    def predict(self, row):
        movie_id = row.movie_id
        user_id = row.user_id
        
        if movie_id not in self.movie_dict:
            return -1
            self.movie_dict[movie_id] = np.random.rand(150)      
        if user_id not in self.user_dict:
            return -1
            self.user_dict[user_id] = np.random.rand(150)
        
        return self.user_dict[user_id].dot(self.movie_dict[movie_id].T)
    
    def update(self, row):
        movie_id = row.movie_id
        user_id = row.user_id
        rating = row.rating
        
        if movie_id not in self.movie_dict or user_id not in self.user_dict:
            return
        
        prediction = self.predict(row)
        error = rating - prediction
        
        
        self.user_dict[user_id] = self.user_dict[user_id] + self.l * (error * self.movie_dict[movie_id] - self.u_reg  * self.user_dict[user_id])
        if self.update_movies:
            self.movie_dict[movie_id] = self.movie_dict[movie_id] + self.l * (error * self.user_dict[user_id] - self.m_reg * self.movie_dict[movie_id])

In [27]:
data_to_send = dict()
for ts in stream_df['timestamp'].unique():
    curr_data = stream_df[stream_df['timestamp'] == ts]
    data_to_send[ts] = curr_data

(apply_list_of_funcs pid=47866) 
(compute_sliced_len pid=47866) 


(compute_sliced_len pid=47864) 
(apply_list_of_funcs pid=47861) 
(apply_list_of_funcs pid=47860) 
(apply_list_of_funcs pid=47865) 
(compute_sliced_len pid=47864) 
(compute_sliced_len pid=47867) 
(apply_list_of_funcs pid=47866) 
(apply_list_of_funcs pid=47865) 


(compute_sliced_len pid=47865) 


(apply_list_of_funcs pid=47866) 
(apply_list_of_funcs pid=47860) 
(compute_sliced_len pid=47859) 
(apply_list_of_funcs pid=47865) 
(apply_list_of_funcs pid=47859) 
(compute_sliced_len pid=47865) 
(compute_sliced_len pid=47866) 
(apply_list_of_funcs pid=47862) 
(compute_sliced_len pid=47863) 
(apply_list_of_funcs pid=47864) 
(apply_list_of_funcs pid=47862) 


(compute_sliced_len pid=47860) 


...

(apply_list_of_funcs pid=48465) 


(compute_sliced_len pid=48465) 


(apply_list_of_funcs pid=48465) 
(apply_list_of_funcs pid=48299) 
(apply_list_of_funcs pid=48408) 
(apply_list_of_funcs pid=48402) 


(apply_list_of_funcs pid=48320) 


(apply_list_of_funcs pid=48308) 


In [36]:
def simulate(scheduler, updater, data, feature_update_cadence, num_process):
    ts_range = sorted(data.keys())
    #ts = df['timestamp'].min()
    #end_ts = df['timestamp'].max()
    
    predictions = []
    next_feature_update_time = ts_range[0] + feature_update_cadence
    num_updates = 0
    num_processed = 0
    
    for ts in ts_range:
        curr_data = data[ts]
        #print(curr_data)
        #print(num_processed)
        for row in curr_data.itertuples():
            scheduler.push(row)
            predictions.append(updater.predict(row))
        num_processed += len(curr_data)
        if ts >= next_feature_update_time:
            # how many records to process
            for i in range(num_process):
                row = scheduler.pop()
                if row:
                    updater.update(row)
                    num_updates += 1
            next_feature_update_time += feature_update_cadence
    '''
    while ts <= end_ts:
        # stream in data
        curr_data = df[df['timestamp'] == ts]
        
        if (num_processed + len(curr_data)) // 10 > num_processed // 10:
            print(num_processed)
        
        for row in curr_data.itertuples():
            scheduler.push(row)
            predictions.append(updater.predict(row))
        
        num_processed += len(curr_data)
        
        if ts == next_feature_update_time:
            # how many records to process
            for i in range(num_process):
                row = scheduler.pop()
                if row:
                    updater.update(row)
                    num_updates += 1
            next_feature_update_time += feature_update_cadence
             
        ts += 1
    '''
    print(num_updates)
    return predictions
    

In [37]:
sched = FIFOScheduler()
updater = SGDUpdater(user_features, product_features, .02, .01, .01)

import time
curr = time.time()
predictions = simulate(sched, updater, data_to_send, 100, 10)
print(time.time - curr)



NameError: name 'FIFOScheduler' is not defined

In [61]:
stream_df['sgd_predictions'] = predictions


,user_id,movie_id,rating,timestamp,prediction,predictions
3,3970,2890,4,973018153,3.356627,3.393901
171,2784,608,5,973021691,3.547614,3.793208
172,2784,1259,5,973021691,3.576579,3.807878
173,2784,2858,5,973021691,3.750439,4.001544
200,2784,3408,3,973021816,3.410957,3.661493
...,...,...,...,...,...,...
500099,4958,2399,1,1046454338,2.397113,2.297596
500100,4958,1407,5,1046454443,3.179136,2.916980
500101,4958,3264,4,1046454548,2.845697,3.169769
500102,4958,2634,3,1046454548,2.887716,2.812283


In [71]:
stream_df.rename(columns={'predictions': 'sgd_predictions'},inplace=True)

In [72]:
stream_df

,user_id,movie_id,rating,timestamp,pre_train_predictions,sgd_predictions
0,2783,1396,4,973018024,3.562985,3.562985
1,2783,2901,5,973018024,3.113821,3.113821
2,3970,3408,4,973018130,3.480977,3.480977
3,3970,2890,4,973018153,3.356627,3.393901
4,2782,2580,4,973018637,-1.000000,-1.000000
...,...,...,...,...,...,...
500099,4958,2399,1,1046454338,2.397113,2.297596
500100,4958,1407,5,1046454443,3.179136,2.916980
500101,4958,3264,4,1046454548,2.845697,3.169769
500102,4958,2634,3,1046454548,2.887716,2.812283


22/03/15 20:42:43 WARN TransportChannelHandler: Exception in connection from amits-mbp.lan/192.168.86.27:64225
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:719)
	at io.netty.channel.nio.NioEventLoop.processSe